# Annotate

In [ ]:
%%writefile toannonated.py
import re
import json
import sys
import urllib
import sys
from HTMLParser import HTMLParser
sys.path.append('../../cgi-bin/')
from wikipedia import *

fileinput = sys.stdin

# import StringIO
# fileinput = StringIO.StringIO(inputstr)
def replacelinks(text):
    hp = HTMLParser()
    
    annotations = []
    deltaStringLength = 0
    hrefreg=r'<a href="([^"]+)">([^>]+)</a>'
    ms = re.finditer(hrefreg, text)
    
    
    for m in ms:     

        url = m.group(1)
        # in the parser, url->encode->quote->encode (does nothing, already unicode)->write to file
        # we already have decoded while reading, we have to encode back, unquote, and decode, so that
        # the last encode on writing can make it back to unicode
        url=url.encode('utf-8')
        url =  urllib.unquote(url)
        url = url.decode('utf-8')
        #sometimes we have [[&]] that has been encoded to &amp; 
        #sometimes we have [[&amp]] that has been encoded to &amp;amp; 
        #so we unexcape twice!
        
        url=hp.unescape(url)
        url=hp.unescape(url)
        url=url.replace(u"\xA0"," ")
        
        x = url.find("#")
        if x!=-1:
            url=url[:x]
        antext = m.group(2)
        if '//' not in url:
            annotations.append({
                "url"    :   url, 
                "surface_form" :   antext, 
                "from"  :   m.start() - deltaStringLength,
                "to"    :   m.start() - deltaStringLength+len(antext)
            })

        deltaStringLength += len(m.group(0)) - len(antext)

    #As a second step, replace all links in the article by their label
    text = re.sub(hrefreg, lambda m:m.group(2), text)  
    return annotations, text
def process():
    hp = HTMLParser()
    rstart=r'<doc id="(.*)" url="(.*)" title="(.*)">'
    rend=r'</doc>'

#     open_ann= open ('open_annotation.json', 'w')
#     ann= open ('annotation.json', 'w')
#     wikitext=open('wikitext.json','w')
    state=0
    page=""
    textlist=[]
    while True:
        line = fileinput.readline()
        if not line:
            break
        line = line.decode('utf-8').strip()
        
        if not line:
            continue
        if re.match(rend,line):
            # If you want to check the title, make sure to encode!
            if textlist and (id2title(wid) is not None):
                opening_ann, opening_text = replacelinks(opening_text)
                ann,text = replacelinks("\n".join(textlist))
                
#                 ElasticSearch   
#                 Buggy: convert annotations to text or do something about it
#                 print json.dumps({"index":{"_type":"page","_id":wid}}, ensure_ascii=False).encode('utf-8')
#                 page={"title": wtitle, "opening_text":opening_text, "opening_annotation":opening_ann,
#                       "text":text, "annotation": ann}
                
#                 print json.dumps(page, ensure_ascii=False).encode('utf-8')
                
#               General
                page={"id":wid, "title": wtitle, "opening_text":opening_text, "opening_annotation":opening_ann,
                      "text":text, "annotation": ann}
                
                print json.dumps(page, ensure_ascii=False).encode('utf-8')
                
            textlist=[]    
            state=0
            continue
        if state==0:
            if re.match('<doc', line):
                ms = re.match(rstart, line)
                wid=ms.group(1)
                wtitle=hp.unescape(ms.group(3)).replace(u"\xA0"," ")

                state = 1
            continue    

        
        if state==1:
            state=2
            continue
        textlist.append(line)
            
        if state==2:
            opening_text=line
            state=3
            continue
            
#     wikitext.close()
#     ann.close()
if __name__ == "__main__": process()

# Extract Anchors

In [1]:
%%writefile extractanchors.py
import json
import sys
import urllib

fileinput = sys.stdin

import StringIO
#fileinput = StringIO.StringIO(inputstr)
while True:
    line = fileinput.readline()
    if not line:
        break
    line = line.decode('utf-8').strip()
    js = json.loads(line)    
    if "annotation" not in js:
        continue
    for a in js["annotation"]:
        print u"{0}\t{1}".format(a["surface_form"],a["url"].replace(" ","_")).encode('utf-8')
                
    

Overwriting extractanchors.py


### extract

`nohup ./WikiExtractor.py -o ~/backup/wikipedia/20160305/texts/xmltexts -b 1000M  -l  --no-templates ~/backup/wikipedia/20160305/original/enwiki-20160305-pages-articles.xml.bz2    >wikiext.log 2>&1 &`

### get annonated
`time cat ~/backup/wikipedia/20160305/texts/xmltexts/wiki_* | python toannonated.py > ~/backup/wikipedia/20160305/texts/enwiki-20160305-annonated.json`

### get anchors

`time cat ~/backup/wikipedia/20160305/texts/enwiki-20160305-annonated.json | python extractanchors.py > ~/backup/wikipedia/20160305/texts/enwiki-20160305-anchor-titles-from-text`



### Augmenting anchors with titles
Examples of titles with no anchors:
Meiluawati
Zdeněk_Svoboda



In [16]:
%%writefile titles2anchors.py

import MySQLdb
_db = MySQLdb.connect(host="127.0.0.1",port=3307,user='root',passwd="emilios",db="enwiki20160305")
_cursor = _db.cursor()
_cursor.execute("select page_title from page where page_namespace=0");
rows = _cursor.fetchall();
for row in rows:
    title=row[0]
    surface_form = title;
    i=surface_form.find('(')
    if i!=-1:
        surface_form = surface_form[:i]
    surface_form = surface_form.replace('_', ' ')
    surface_form = surface_form.strip()
    
    print surface_form+"\t"+title
    

Overwriting titles2anchors.py


### Titles to Anchors
`time python titles2anchors.py > ~/backup/wikipedia/20160305/texts/enwiki-20160305-anchor-from-titles.tsv`

`cp enwiki-20160305-anchor-titles-from-text.tsv  enwiki-20160305-anchor-titles.tmp.tsv`
`cat enwiki-20160305-anchor-from-titles.tsv >> enwiki-20160305-anchor-titles.tmp.tsv`


### modify anchors
`time cat enwiki-20160305-anchor-titles.tmp.tsv | sort| uniq -c | sed 's/^ *//;s/ /\t/' > enwiki-20160305-anchor-titles.tsv`


# Resolving redirects    

In [ ]:
%%writefile resolveredirs.py
import sys
sys.path.append('../../cgi-bin/')
from collections import defaultdict
import imp
import json
import urllib
from wikipedia import *
fileinput = sys.stdin
errout = sys.stderr
import StringIO
#fileinput = StringIO.StringIO(inputstr)
pairdict = defaultdict(int)
while True:
    line = fileinput.readline()
    if not line:
        break
    line = line.strip()
    tp = line.split("\t")
    if len(tp) !=3:
        errout.write("Line Error: " + line)
        continue
    freq,anchor,title=tp
    wid=title2id(title)
    if wid is None:
        title=title[0].upper()+title[1:]
    wid=title2id(title)
    if wid is None:
        errout.write(anchor+"\t"+title+"\t"+str(freq)+"\n")
        continue
    pairdict[(anchor,wid)] += int(freq)
for (anchor,wid),freq in pairdict.iteritems():
    print anchor+"\t"+str(wid)+"\t"+str(freq)

# Applying resolves

`time cat ~/backup/wikipedia/20160305/texts/enwiki-20160305-anchor-titles.tsv | python resolveredirs.py > ~/backup/wikipedia/20160305/texts/enwiki-20160305-anchor-ids.tsv 2> ~/backup/wikipedia/20160305/texts/enwiki-20160305-anchor-ids.dead.tsv`

### importing to mysql
`bash importall ~/backup/wikipedia/20160305/texts 20160305 root emilios`

### extracting anchors lists
cat enwiki-20160305-anchor-ids.tsv| cut -f1 -d$'\t'| sort | uniq > enwiki-20160305-anchors.tsv



# ToSolr v2. converts the annotations to a jsonstring

In [ ]:
%%writefile tosolr.py
import json
import sys
import os

fileinput = sys.stdin

import StringIO
#fileinput = StringIO.StringIO(inputstr)

n=1000
count=0;
f=None
os.mkdir("chunks")
seperator=''
firstline=None
while True:
    line = fileinput.readline()
    if not line:
        break
        
    line = line.decode('utf-8').strip()
    if not line:
        continue
        
    if count%n==0:
        if f is not None:
            f.write('\n]\n')
            f.close()
        f=open('chunks/chunk'+str(int(count/n)).zfill(10)+'.json','w')    
        f.write('[\n')
        firstline=True
        seperator=""
    count +=1
    
    page=json.loads(line)
    page["annotation"]=json.dumps(page["annotation"], ensure_ascii=False)
    page["opening_annotation"]=json.dumps(page["opening_annotation"], ensure_ascii=False)
    f.write(seperator+json.dumps(page, ensure_ascii=False).encode('utf-8'))
    if firstline:
        firstline=False
        seperator=",\n"
    
    
if f is not None:
    f.write('\n]\n')
    f.close()
    

    
    

## Preparing solr importable
`time python tosolr.py <~/backup/wikipedia/20160305/texts/enwiki-20160305-annonated.json`

## creating solr nodes

`bin/solr create -c enwiki20160305`

`bin/solr stop`

`cp ~/backup/projects/wikifier/preparation_script/solrsettings/conf/* .`

`bin/solr start`

## importing to solr
`nohup time bash loadwiki >load.txt 2>&1 &`

**count**
curl 'http://localhost:8983/solr/enwiki20160305/select?indent=on&q=*:*&rows=0&wt=json'

# tager preparation

`bin/solr create -c enwikianchors20160305`

`bin/solr stop`

`cp -r ~/backup/projects/wikisim/preparation_script/text/solrtagger/lib/ server/solr/`


`cp ~/backup/projects/wikifier/preparation_script/solrtagger/conf/* server/solr/enwikianchors20160305/conf/`


`curl -X POST --data-binary @enwiki-20160305-anchors.tsv.txt -H 'Content-type:application/csv' \
  'http://localhost:8983/solr/enwikianchors20160305/update?commit=true&optimize=true&separator=%09&encapsulator=%00&fieldnames=name'`
  
  


# Random Access to paragraphs with links

In [ ]:
import re
import random
qstr = 'http://localhost:8983/solr/enwiki20160305/select'
q='title:Bruce_Lee'
params={'indent':'on', 'wt':'json', 'q':q}
r = requests.get(qstr, params=params)
r =  r.json()['response']['docs'][0]

pos=[m.start() for m in re.finditer('\n', r['text'])]    
n = index = random.randint(0,len(pos)-1)
s = pos[n]
e = pos[n+1] if n < len(pos)-1 else len(r['text'])
par = r['text'][s:e]
print r['title']
print "par no:",n, par[:10],"...", par[len(par)-10:]
for ann in json.loads(r['annotation']):
    #print ann
    if int(ann['from']) < s: 
        continue
    if int(ann['to']) >= e: 
        break
    surfintext = r['text'][ann['from']:ann['to']]
    print "detected: %s, in annotation: %s, title: %s" % (surfintext, ann[u'surface_form'], ann['url'])
        